In [18]:
import numpy as np
import pandas as pd
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_rows = 20
np.set_printoptions(precision=4, suppress=True)

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## 11.4 시간대 다루기

📌 파이썬의 시간대에 대해 알아보기(datetime.timezone) : [link](https://spoqa.github.io/2019/02/15/python-timezone.html)

![utc](https://upload.wikimedia.org/wikipedia/commons/8/88/World_Time_Zones_Map.png)

- 파이썬에서 시간대 정보는 전 세계 시간대 정보를 모아둔 [올슨 데이터베이스](https://en.wikipedia.org/wiki/Tz_database)를 담고 있는 서드파티 라이브러리인 `pytz`에서 얻어 온다.
- `pandas`는 `pytz`의 기능을 사용하고 있으므로 시간대 이름 외에 API의 다른 부분은 무시해도 상관없다. [시간대 이름](https://gist.github.com/heyalexej/8bf688fd67d7199be4a1682b3eec7568)은 문서와 파이썬 셀에서 직접 확인할 수 있다.
- `pandas`의 메서드에서는 시간대 이름이나 시간대 객체를 모두 사용할 수 있지만 <u>시간대 이름을 사용하기를 권장</u>한다.

In [2]:
import pytz
pytz.common_timezones[-5:]

['US/Eastern', 'US/Hawaii', 'US/Mountain', 'US/Pacific', 'UTC']

In [8]:
# 모든 시간대 이름
pytz.all_timezones[:5]

['Africa/Abidjan',
 'Africa/Accra',
 'Africa/Addis_Ababa',
 'Africa/Algiers',
 'Africa/Asmara']

In [3]:
# 시간대 객체를 얻으려면 pytz.timezon을 사용하면 된다.
tz = pytz.timezone('America/New_York')
tz

<DstTzInfo 'America/New_York' LMT-1 day, 19:04:00 STD>

### 11.4.1 시간대 지역화와 변환

기본적으로 `pandas`에서 시계열은 **시간대를 엄격히 다루지 않는다.**

#### pandas와 timezone

✨ remind : date_range<br>

`pandas.date_range`를 사용하여 특정 빈도에 따라 지정한 길이만큼의 `DatetimeIndex`를 생성한다. 교재에서는 436 페이지에 등장!

```python
pd.date_range(start=None, end=None, periods=None, freq=None, tz=None, normalize=False, name=None, closed=None, **kwargs)
```

In [9]:
rng = pd.date_range('3/9/2012 9:30', periods=6, freq='D')
ts = pd.Series(np.random.randn(len(rng)), index=rng)
ts

2012-03-09 09:30:00   -0.204708
2012-03-10 09:30:00    0.478943
2012-03-11 09:30:00   -0.519439
2012-03-12 09:30:00   -0.555730
2012-03-13 09:30:00    1.965781
2012-03-14 09:30:00    1.393406
Freq: D, dtype: float64

In [11]:
# 색인의 tz 필드는 None이다.
print(ts.index.tz)

None


`tz` 파라미터를 이용하여 시간대를 지정해서 날짜 범위를 생성할 수 있다.

In [14]:
pd.date_range('3/9/2012 9:30', periods=10, freq='D', tz='UTC')

DatetimeIndex(['2012-03-09 09:30:00+00:00', '2012-03-10 09:30:00+00:00',
               '2012-03-11 09:30:00+00:00', '2012-03-12 09:30:00+00:00',
               '2012-03-13 09:30:00+00:00', '2012-03-14 09:30:00+00:00',
               '2012-03-15 09:30:00+00:00', '2012-03-16 09:30:00+00:00',
               '2012-03-17 09:30:00+00:00', '2012-03-18 09:30:00+00:00'],
              dtype='datetime64[ns, UTC]', freq='D')

#### 지역화
**지역화** 시간으로의 변환은 `tz_localize` 메서드로 처리할 수 있다.

In [21]:
ts
ts_utc = ts.tz_localize('UTC')
ts_utc

2012-03-09 09:30:00   -0.204708
2012-03-10 09:30:00    0.478943
2012-03-11 09:30:00   -0.519439
2012-03-12 09:30:00   -0.555730
2012-03-13 09:30:00    1.965781
2012-03-14 09:30:00    1.393406
Freq: D, dtype: float64

2012-03-09 09:30:00+00:00   -0.204708
2012-03-10 09:30:00+00:00    0.478943
2012-03-11 09:30:00+00:00   -0.519439
2012-03-12 09:30:00+00:00   -0.555730
2012-03-13 09:30:00+00:00    1.965781
2012-03-14 09:30:00+00:00    1.393406
Freq: D, dtype: float64

In [22]:
ts.index
ts_utc.index

DatetimeIndex(['2012-03-09 09:30:00', '2012-03-10 09:30:00',
               '2012-03-11 09:30:00', '2012-03-12 09:30:00',
               '2012-03-13 09:30:00', '2012-03-14 09:30:00'],
              dtype='datetime64[ns]', freq='D')

DatetimeIndex(['2012-03-09 09:30:00+00:00', '2012-03-10 09:30:00+00:00',
               '2012-03-11 09:30:00+00:00', '2012-03-12 09:30:00+00:00',
               '2012-03-13 09:30:00+00:00', '2012-03-14 09:30:00+00:00'],
              dtype='datetime64[ns, UTC]', freq='D')

#### 시간대 변환
시계열이 특정 시간대로 지역화되고 나면 `tz_convert`를 이용해서 다른 시간대로 변환 가능하다.

In [23]:
# America/New_York은 서머 타임이 적용되고 있다.
ts_utc.tz_convert('America/New_York')

2012-03-09 04:30:00-05:00   -0.204708
2012-03-10 04:30:00-05:00    0.478943
2012-03-11 05:30:00-04:00   -0.519439
2012-03-12 05:30:00-04:00   -0.555730
2012-03-13 05:30:00-04:00    1.965781
2012-03-14 05:30:00-04:00    1.393406
Freq: D, dtype: float64

In [33]:
# America/New_York → EST → UTC 또는 Europe/Berlin
ts_newyork = ts.tz_localize('America/New_York')
ts_newyork.index
ts_newyork.tz_convert('EST').index
ts_newyork.tz_convert('EST').tz_convert('UTC').index
ts_newyork.tz_convert('EST').tz_convert('Europe/Berlin').index

DatetimeIndex(['2012-03-09 09:30:00-05:00', '2012-03-10 09:30:00-05:00',
               '2012-03-11 09:30:00-04:00', '2012-03-12 09:30:00-04:00',
               '2012-03-13 09:30:00-04:00', '2012-03-14 09:30:00-04:00'],
              dtype='datetime64[ns, America/New_York]', freq=None)

DatetimeIndex(['2012-03-09 09:30:00-05:00', '2012-03-10 09:30:00-05:00',
               '2012-03-11 08:30:00-05:00', '2012-03-12 08:30:00-05:00',
               '2012-03-13 08:30:00-05:00', '2012-03-14 08:30:00-05:00'],
              dtype='datetime64[ns, EST]', freq=None)

DatetimeIndex(['2012-03-09 14:30:00+00:00', '2012-03-10 14:30:00+00:00',
               '2012-03-11 13:30:00+00:00', '2012-03-12 13:30:00+00:00',
               '2012-03-13 13:30:00+00:00', '2012-03-14 13:30:00+00:00'],
              dtype='datetime64[ns, UTC]', freq=None)

DatetimeIndex(['2012-03-09 15:30:00+01:00', '2012-03-10 15:30:00+01:00',
               '2012-03-11 14:30:00+01:00', '2012-03-12 14:30:00+01:00',
               '2012-03-13 14:30:00+01:00', '2012-03-14 14:30:00+01:00'],
              dtype='datetime64[ns, Europe/Berlin]', freq=None)

`tz_localize`와 `tz_convert`는 모두 `DatetimeIndex`의 인스터스 메서드이다.

In [34]:
ts.index.tz_localize('Asia/Shanghai')

DatetimeIndex(['2012-03-09 09:30:00+08:00', '2012-03-10 09:30:00+08:00',
               '2012-03-11 09:30:00+08:00', '2012-03-12 09:30:00+08:00',
               '2012-03-13 09:30:00+08:00', '2012-03-14 09:30:00+08:00'],
              dtype='datetime64[ns, Asia/Shanghai]', freq=None)

### 11.4.2 시간대를 고려해서 Timestamp 객체 다루기
시계열이나 날짜 범위와 비슷하게 개별 timestamp 객체도 시간대를 고려한 형태로 변환이 가능하다.

In [37]:
stamp = pd.Timestamp('2011-03-12 04:00')
stamp

stamp_utc = stamp.tz_localize('utc')
stamp_utc

stamp_utc.tz_convert('America/New_York')

Timestamp('2011-03-12 04:00:00')

Timestamp('2011-03-12 04:00:00+0000', tz='UTC')

Timestamp('2011-03-11 23:00:00-0500', tz='America/New_York')

Timestamp 객체를 생성할 때 시간대를 `tz` 파라미터를 사용하여 시간대를 직접 넘겨주는 것도 가능하다.

In [38]:
stamp_moscow = pd.Timestamp('2011-03-12 04:00', tz='Europe/Moscow')
stamp_moscow

Timestamp('2011-03-12 04:00:00+0300', tz='Europe/Moscow')

시간대를 고려한 Timestamp 객체는 내부적으로 UTC 타임스탬프 값을 유닉스 에포스(Unix epoch, 1970년 1월 1일)부터 현재까지의 나노초로 저장하고 있다. 이 UTF 값은 시간대 변환 과정에서 변하지 않고 유지된다.

In [39]:
stamp_utc.value
stamp_utc.tz_convert('America/New_York').value

1299902400000000000

1299902400000000000

`pandas`의 `DateOffset` 객체를 이용해서 시간 연산을 수행할 때는 가능하다면 서머타임을 고려한다.

In [56]:
from pandas.tseries.offsets import Hour

# 2012년도 서머타임 시작일 : 3월 11일
# 2012년도 서머타임 종료일 : 11월 4일

# DST로 전환되기 직전(30분 전)의 Timestamp 생성
stamp = pd.Timestamp('2018-03-11 01:30', tz='US/Eastern')
stamp
stamp + Hour()

Timestamp('2018-03-11 01:30:00-0500', tz='US/Eastern')

Timestamp('2018-03-11 03:30:00-0400', tz='US/Eastern')

In [57]:
# DST로 전환되기 90분 전의 Timestamp 생성
stamp = pd.Timestamp('2012-11-04 00:30', tz='US/Eastern')
stamp
stamp + 2 * Hour()

Timestamp('2012-11-04 00:30:00-0400', tz='US/Eastern')

Timestamp('2012-11-04 01:30:00-0500', tz='US/Eastern')

### 11.4.3 다른 시간대 간의 연산
서로 다른 시간대를 갖는 두 시계열이 하나로 합쳐지면 결과는 UTC가 된다.<br>
타임스탬프를 내부적으로 UTC로 저장되므로 추가적인 변환이 불필요한 명료한 연산이다.

In [59]:
rng = pd.date_range('3/7/2012 9:30', periods=10, freq='B')
ts = pd.Series(np.random.randn(len(rng)), index=rng)
ts

2012-03-07 09:30:00    1.007189
2012-03-08 09:30:00   -1.296221
2012-03-09 09:30:00    0.274992
2012-03-12 09:30:00    0.228913
2012-03-13 09:30:00    1.352917
2012-03-14 09:30:00    0.886429
2012-03-15 09:30:00   -2.001637
2012-03-16 09:30:00   -0.371843
2012-03-19 09:30:00    1.669025
2012-03-20 09:30:00   -0.438570
Freq: B, dtype: float64

In [62]:
ts1 = ts[:7].tz_localize('Europe/London')
ts2 = ts1[2:].tz_convert('Europe/Moscow')
result = ts1 + ts2
result.index

DatetimeIndex(['2012-03-07 09:30:00+00:00', '2012-03-08 09:30:00+00:00',
               '2012-03-09 09:30:00+00:00', '2012-03-12 09:30:00+00:00',
               '2012-03-13 09:30:00+00:00', '2012-03-14 09:30:00+00:00',
               '2012-03-15 09:30:00+00:00'],
              dtype='datetime64[ns, UTC]', freq=None)